A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column                   | Description                                                                      |
|------------------------- |--------------------------------------------------------------------------------- |
| `student_id`             | A unique ID for each student.                                                    |
| `city`                   | A code for the city the student lives in.                                        |
| `city_development_index` | A scaled development index for the city.                                         |
| `gender`                 | The student's gender.                                                            |
| `relevant_experience`    | An indicator of the student's work relevant experience.                          |
| `enrolled_university`    | The type of university course enrolled in (if any).                              |
| `education_level`        | The student's education level.                                                   |
| `major_discipline`       | The educational discipline of the student.                                       |
| `experience`             | The student's total work experience (in years).                                  |
| `company_size`           | The number of employees at the student's current employer.                       |
| `company_type`           | The type of company employing the student.                                       |
| `last_new_job`           | The number of years between the student's current and previous jobs.             |
| `training_hours`         | The number of hours of training completed.                                       |
| `job_change`             | An indicator of whether the student is looking for a new job (`1`) or not (`0`). |

&nbsp;

&nbsp;

***
The Head Data Scientist at Training Data Ltd. has asked you to create a DataFrame called ds_jobs_transformed that stores the data in customer_train.csv much more efficiently. Specifically, they have set the following requirements:

- Columns containing categories with only two factors must be stored as Booleans (bool).
- Columns containing integers only must be stored as 32-bit integers (int32).
- Columns containing floats must be stored as 16-bit floats (float16).
- Columns containing nominal categorical data must be stored as the category data type.
- Columns containing ordinal categorical data must be stored as ordered categories, and not mapped to numerical values, with an order that reflects the natural order of the column.
- The DataFrame should be filtered to only contain students with 10 or more years of experience at companies with at least 1000 employees, as their recruiter base is suited to more experienced professionals at enterprise companies.
- If you call .info() or .memory_usage() methods on ds_jobs and ds_jobs_transformed after you've preprocessed it, you should notice a substantial decrease in memory usage.

In [114]:
# Import necessary libraries
import pandas as pd
import numpy as np
from pandas.api.types import CategoricalDtype

# Load the dataset
ds_jobs = pd.read_csv("customer_train.csv")

# View the dataset
ds_jobs.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevant experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevant experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


In [115]:
# Create a copy of ds_jobs for transforming
ds_jobs_transformed = ds_jobs.copy()

# Start coding here. Use as many cells as you like!
pd.options.display.max_columns = 100

ds_jobs_transformed['student_id'] = ds_jobs_transformed['student_id'].astype(np.int32)
ds_jobs_transformed['training_hours'] = ds_jobs_transformed['training_hours'].astype(np.int32)
ds_jobs_transformed['city_development_index'] = ds_jobs_transformed['city_development_index'].astype(np.float16)
ds_jobs_transformed['city'] = ds_jobs_transformed['city'].astype('category')
ds_jobs_transformed['enrolled_university'].fillna('no_enrollment', inplace=True)
eu_cat = CategoricalDtype(categories=['no_enrollment', 'Part time course', 'Full time course'], ordered=True)
ds_jobs_transformed['enrolled_university'] = ds_jobs_transformed['enrolled_university'].astype(eu_cat)

# gender - category
ds_jobs_transformed['gender'].fillna('Other', inplace=True)
ds_jobs_transformed['gender'] = ds_jobs_transformed['gender'].astype('category')

# relevant_experience - category
ds_jobs_transformed['relevant_experience'].replace({'Has relevant experience':1, 'No relevant experience':0}, inplace=True)
ds_jobs_transformed['relevant_experience'] = ds_jobs_transformed['relevant_experience'].astype(bool)

# education_level - category?
ds_jobs_transformed['education_level'].fillna('Other', inplace=True)
el_cat = CategoricalDtype(categories=['Other', 'Primary School', 'High School', 'Graduate', 'Masters', 'Phd'], ordered=True)
ds_jobs_transformed['education_level'] = ds_jobs_transformed['education_level'].astype(el_cat)

# major_discipline - category?
ds_jobs_transformed['major_discipline'].fillna('Other', inplace=True)
ds_jobs_transformed['major_discipline'].replace({'No Major':'Other'}, inplace=True)
ds_jobs_transformed['major_discipline'] = ds_jobs_transformed['major_discipline'].astype('category')

# experience - int, >= 10
ds_jobs_transformed['experience'].fillna('0', inplace=True)
exp_cat = CategoricalDtype(categories=['<1','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','>20'], ordered=True)
ds_jobs_transformed['experience'] = ds_jobs_transformed['experience'].astype(exp_cat)
ds_jobs_transformed = ds_jobs_transformed[ds_jobs_transformed['experience'] >= '10']

# company_size - ordinal, >=1000
ds_jobs_transformed.dropna(subset='company_size', inplace=True)
com_size_cat = CategoricalDtype(categories=['<10','10-49','50-99','100-499','500-999','1000-4999','5000-9999','10000+'], ordered=True)
ds_jobs_transformed['company_size'] = ds_jobs_transformed['company_size'].astype(com_size_cat)
ds_jobs_transformed = ds_jobs_transformed[ds_jobs_transformed['company_size'] >= '1000-4999']

# company_type - category
ds_jobs_transformed['company_type'].fillna('Other', inplace=True)
ds_jobs_transformed['company_type'] = ds_jobs_transformed['company_type'].astype('category')

# last_new_job - int?
ds_jobs_transformed['last_new_job'].fillna('0', inplace=True)
ds_jobs_transformed['last_new_job'].replace({'never':'0'}, inplace=True)
lnj_cat = CategoricalDtype(categories=['0','1','2','3','4','>4'], ordered=True)
ds_jobs_transformed['last_new_job'] = ds_jobs_transformed['last_new_job'].astype(lnj_cat)

# job_change - int?
ds_jobs_transformed['job_change'] = ds_jobs_transformed['job_change'].astype(bool)

print(f"\nds_jobs: {ds_jobs.info()}\n")
print(f"\nds_jobs_transformed: {ds_jobs_transformed.info()}\n")
print(f"\nds_jobs: {ds_jobs.memory_usage()}\n")
print(f"\nds_jobs_transformed: {ds_jobs_transformed.memory_usage()}\n")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevant_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  job_change              19158 non-null  float64
dtypes: float64(2), int64(2), object(10)
me